In [1]:
import numpy as np
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.colors import groups
import math
from bokeh.io import export_png
import wandb



In [2]:
 def unit_poly_verts(theta, centre):
        """Return vertices of polygon for subplot axes.
        This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
        """
        x0, y0, r = [centre ] * 3
        verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
        return verts

def radar_patch(r, theta, centre ):
    """ Returns the x and y coordinates corresponding to the magnitudes of 
    each variable displayed in the radar plot
    """
    # offset from centre of circle
    offset = 0.0
    yt = (r*centre + offset) * np.sin(theta) + centre 
    xt = (r*centre + offset) * np.cos(theta) + centre 
    return xt, yt
        

def get_radar_plot(metrics, values, color_group, size=256, title=None):
    """ Generates one radar plot:
    metrics: List of names for each vertices
    values: Corresponding scores
    color_group: A group in the sense of Bokeh.colors.groups
    size: the size of the chart
    title: Facultative title of the graph
    
    """
    num_vars = 4
    centre = 0.5
    
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2
    
   
    
    verts = unit_poly_verts(theta, centre)
    x = np.array([v[0] for v in verts])
    y = np.array([v[1] for v in verts])
    x[3] += .1
    x[1] -= .1
    y[2] -= .08 # Manually adjusting the positioning of each labels. Not super great.
    p = figure(title=title, match_aspect=True, x_range=(-.2, 1.2), y_range=(-.2, 1.2), width=size, height=size, toolbar_location=None)
    
    source = ColumnDataSource({'x':x,'y':y,'text':metrics})
    
    p.ellipse(x=0.5, y=0.5, width=1., height=1., fill_color='gray', fill_alpha=0.05, line_color=None)
    p.line(x=[0, 1], y=[.5, .5], line_dash='dotted', color='black') # Center horizontal line
    p.line(y=[0, 1], x=[.5, .5], line_dash='dotted', color='black') # Center vertical line
    p.circle(x=[0, .5, 1, .5, .5], y=[.5,1,.5,0, .5], color='black') # Little black dots for each labels

    base_color =color_group[0].lighten(0.5)
    xt, yt = radar_patch(values, theta, centre)
    print()
    for index in range(4):
        c1 = xt[index], yt[index]
        p_next = (index+1)%4
        c2 = xt[p_next], yt[p_next]
        xs = [c1[0], c2[0], 0.5]
        ys = [c1[1], c2[1], 0.5]
        p.patch(x=xs, y=ys, fill_color=color_group[index].darken(0.1), line_color=base_color, alpha=0.75, line_width=2)
        
    labels = LabelSet(x="x",y="y",text="text",source=source, text_align='center')
    p.add_layout(labels)
    labels = [f'{v:.1%}' for v in values] # Indicates the score
    source_new = ColumnDataSource({'x':xt,'y':yt,'text':labels})
    labels_score = LabelSet(x="x",y="y",text="text",source=source_new, text_align='center')
    p.add_layout(labels_score)

    p.grid.visible = False
    p.axis.visible = False
    return p

In [3]:
api = wandb.Api()
run = api.run("liv4d-polytechnique/MessidorValidation/3kz0fvji")

In [4]:
labels = ["Exudates",
            "Hemorrhages",
            "Microaneurysms",
            "Soft Exudates",
            "Drusen",
            "Neovascularization",
            "OpticDisk",
            "Macula",
            "Vessels"]

metrics = ["Accuracy", "IoU", "Precision", "AUC Precision Recall"]
# Could also include Specificty, Recall
all_columns = [f'{metric} {lesion}'  for lesion in labels for metric in metrics]
scores = run.history(keys=all_columns)

In [5]:
all_plots = []
cgroups = [groups.blue, groups.brown, groups.cyan, groups.green, groups.orange, groups.pink, groups.purple, groups.red, groups.yellow, groups.black]

nb_col = 3 # Only matters for grid display next cell.
nb_row = math.ceil(len(labels)/nb_col)
for irow in range(nb_row):
    row = []
    for icol in range(nb_col):
        index = nb_col*irow + icol
        if index>=len(labels):
            row.append(None)
        else:
            label = labels[index]
            columns = [f'{m} {label}' for m in metrics]
            values = np.array(scores[columns].iloc[0])
            label_metrics = [m.replace("AUC Precision Recall", "Avg. Pr.") for m in metrics]
            plot = get_radar_plot(label_metrics, values, cgroups[index], size=450)
            plot.background_fill_color = None
            plot.border_fill_color = None
            export_png(plot, filename=f"results/{label}.png") 
            row.append(plot)
    all_plots.append(row)

In [6]:
from bokeh.layouts import grid
output_notebook()
p = grid(all_plots)
show(p)

Loading BokehJS ...